In [ ]:
!wget https://raw.githubusercontent.com/adityatripathiiit/temporary/main/continent.csv
!wget https://raw.githubusercontent.com/adityatripathiiit/temporary/main/energy_use_per_person.csv
!wget https://raw.githubusercontent.com/adityatripathiiit/temporary/main/gdppercapita_us_inflation_adjusted.csv
!wget https://raw.githubusercontent.com/adityatripathiiit/temporary/main/population_total.csv

**Importing Libraries**

In [7]:
import plotly.express as px 
import numpy as np
import pandas as pd

**Importing the CSV data**


In [8]:
energy_data = pd.read_csv("energy_use_per_person.csv")
gdp_data = pd.read_csv("gdppercapita_us_inflation_adjusted.csv")
population_data = pd.read_csv("population_total.csv").dropna().set_index("country")
continent_data = pd.read_csv("continent.csv").dropna().set_index("Country_Name")

**Filtering the data from 1971-2013 and Removing Uncommon Rows/Columns from the dataframe**

In [9]:
for i in energy_data.columns:
    if(i != "country" and (int(i) < 1971 or int(i) >2013)):
        del energy_data[i]
for i in gdp_data.columns:
    if(i != "country" and (int(i) < 1971 or int(i) >2013)):
        del gdp_data[i]
energy_data = energy_data.dropna()
gdp_data = gdp_data.dropna()
common_countries = np.intersect1d(energy_data["country"].tolist(), gdp_data["country"].tolist())
drop_energy =  []
drop_gdp = []
for i in energy_data["country"].tolist():
    if i not in common_countries:
        drop_energy.append(i)
energy_data = energy_data.set_index("country")
energy_data.drop(drop_energy, inplace = True)
for i in gdp_data["country"].tolist():
    if i not in common_countries:
        drop_gdp.append(i)
gdp_data = gdp_data.set_index("country")
gdp_data.drop(drop_gdp, inplace = True)

**Initializing the Plot variables**

In [10]:
plot_data = {"country":[], "GDP": [], "Energy":[],"population": [], "Year":[],"Continent_Name": [] }
for year in range(1971, 2014):
    for country in common_countries: 
        plot_data["country"].append(country)
        plot_data["GDP"].append(gdp_data.loc[country, str(year)])
        plot_data["Energy"].append(energy_data.loc[country, str(year)])
        if (population_data.loc[country, str(year)] > 326000000):
            plot_data["population"].append(356000000)
        else:
            plot_data["population"].append(population_data.loc[country, str(year)])
        
        # plot_data["population"].append(1)
        plot_data["Year"].append(str(year))
        if (country in continent_data.index):
            plot_data["Continent_Name"].append(continent_data.loc[country, "Continent_Name"])
        else:
            plot_data["Continent_Name"].append("Unknown")

**Plotting the animation**

In [11]:
fig = px.scatter(plot_data,x="GDP", y="Energy",animation_frame="Year", 
 animation_group="country",size="population", 
 color="Continent_Name",
 log_x=True,
 size_max=50,
 hover_name="country"
 )
# Tune marker appearance and layout
fig.update_traces(mode='markers', marker=dict(sizemode='area'))
fig.update_layout(
 title= 'Energy Consumption per Capita v. GDP Per Capita, 1971~2013',
 xaxis=dict(
 title='GDP per Capita',
 gridcolor='white',
 gridwidth=2,
 ),
 yaxis=dict(
 title='Energy Consumption',
 gridcolor='white',
 gridwidth=2,
 ),
 paper_bgcolor='rgb(243, 243, 243)',
 plot_bgcolor='rgb(243, 243, 243)',
)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 600
fig.show()